<h2 style='text-align:center;'>Model</h2>

<hr>

In [78]:
# imprt libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 

In [79]:
#Data Preprocessing

def preprocess(dataset:pd.DataFrame) -> pd.DataFrame :
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [80]:
# Feature Engineering

def FeatureEngineering(dataset:pd.DataFrame,
                       categorical_variables:list,
                       numerical_variables:list,
                       isTrain:bool,
                       scaleColumns:list):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()
    df = df[numerical_important + categorical_important]

    # we create a folder to save our data there 
    path = os.getcwd()
    assets_path = path+'\\assets'
    if not os.path.exists(assets_path):
        os.makedirs(assets_path)

    # Dummy variables
    if isTrain:
        def ohe(dataset:pd.DataFrame, categorical_columns:str):
            df = dataset.copy()
            df_info = pd.DataFrame(columns=['col', 'uniques'])
            for col in categorical_columns:
                list_Uniques = df[col].unique()
                df_info.loc[len(df_info)] = [col, ','.join(map(str, list_Uniques))]
                for name in list_Uniques:
                    df[str(col)+'_'+str(name)] = df[col].apply(lambda x : 1 if x==name else 0)
                del df[col]

            df_info.to_csv(assets_path+'\\ohe_info.csv', index=False)
            return df

        df=ohe(df, categorical_important)
    else:
        df_info = pd.read_csv(assets_path+'\ohe_info.csv')
        for i in range(len(df_info)):
            col = df_info.iloc[i,:].col
            list_Uniques = df_info.iloc[i,:].uniques.split(',')
            for name in list_Uniques:
                df[str(col)+'_'+str(name)] =  df[col].apply(lambda x : 1 if x==name else 0)
            del df[col]

    # Feature scaling
    if isTrain:
        scale_info = pd.DataFrame(columns=['col','mean','std'])
        for col in scaleColumns:
            mean_col = df[col].mean()
            std_col = df[col].std()
            df[col] = (df[col]-mean_col)/std_col
            scale_info.loc[len(scale_info)] = [col, mean_col, std_col]
        
        scale_info.to_csv(assets_path+'\\scale_info.csv', index=False)
    else:
        scale_info = pd.read_csv(assets_path+'\\scale_info.csv')
        
        for i in range(len(scale_info)):
            col = scale_info.loc[i,'col']
            mean_col = scale_info.loc[i,'mean']
            std_col = scale_info.loc[i,'std']

            df[col] = (df[col]-mean_col)/std_col
    return df

In [81]:
df = pd.read_csv('Churn_prediction.csv')
df = preprocess(df)

categorical_important=['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='churn'],
                                                    df['churn'],
                                                    test_size=0.2,
                                                    random_state=1)

<hr>

In [83]:
X_train = preprocess(X_train)
X_train = FeatureEngineering(X_train,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=True,
                            scaleColumns=numerical_important
                            )

<hr>

In [84]:
from sklearn.model_selection import KFold 
from sklearn.metrics import roc_auc_score

def crossValidation(X:pd.DataFrame, y:pd.DataFrame, model, nFold:int=5):
    X_train = X.copy()
    y_train = y.copy()
    Model = model
    kfold= KFold(n_splits=nFold, shuffle=True, random_state=2)
    scores = []
    for train_idx, val_idx in kfold.split(X_train):
        df_train_kf = X_train.iloc[train_idx]
        df_val_kf = X_train.iloc[val_idx]
    
        y_train_kf = y_train.iloc[train_idx]
        y_val_kf = y_train.iloc[val_idx]
    
        Model.fit(df_train_kf, y_train_kf)
        pred = Model.predict(df_val_kf)
        scores.append(roc_auc_score(y_val_kf, pred))
    
    return scores

<hr>

<h2>Logistic Regression</h2>

In [85]:
from sklearn.linear_model import LogisticRegression
clf_LogisticRegression = LogisticRegression(solver='liblinear', random_state=0)

scores = crossValidation(X=X_train,y=y_train,model=clf_LogisticRegression, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.7 std:0.02


<hr>

<h2>K Nearest Neighbors</h2>

In [86]:
from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier()

scores = crossValidation(X=X_train,y=y_train,model=clf_KNN, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.7 std:0.02


<hr>

<h2>Naive Bayes</h2>

In [93]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB() 

scores = crossValidation(X=X_train, y=y_train, model=clf_NB, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.74 std:0.02


<hr>

<h2>Support Vector machine for Classification (SVC)</h2>

In [88]:
from sklearn.svm import SVC
clf_SVC = SVC(kernel='linear', decision_function_shape ='ovo', random_state=6)

scores = crossValidation(X=X_train, y=y_train, model=clf_SVC, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.7 std:0.02


<hr>

<h2>Desicion Tree Classification</h2>

In [89]:
from sklearn.tree import DecisionTreeClassifier
clf_DT = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=1)

scores = crossValidation(X=X_train, y=y_train, model=clf_DT, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.71 std:0.02


<hr>

<h2>Random Forest</h2>

In [90]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(n_estimators=10, criterion='entropy', max_depth=7, random_state=6)

scores = crossValidation(X=X_train,y=y_train,model=clf_RF, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.7 std:0.02


<hr>

<h2>Gradiant Boosting model</h2>

In [91]:
from sklearn.ensemble import GradientBoostingClassifier
clf_GB = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3)

scores = crossValidation(X=X_train,y=y_train,model=clf_GB, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.68 std:0.02


<hr>

<h2>XGBoost</h2>

In [92]:
from xgboost import XGBClassifier
clf_xgboost = XGBClassifier()

scores = crossValidation(X=X_train,y=y_train,model=clf_xgboost, nFold=10)
print('roc_auc_score mean:{0} std:{1}'.format(round(np.mean(scores),2), round(np.std(scores),2)))

roc_auc_score mean:0.69 std:0.02


<hr>

Naive Bayes is the best model so I choose it and tune the parameters of this model.